<center>
  <font size="7">Tests unitaires de l'API</font><br>
  <font size="5">Projet 7 - Implémentez un modèle de scoring</font>
</center>
<div align="right">
  <font size="4"><i>par Jean Vallée</i></font>
</div>

<hr size=5>

# Scénario de test unitaire 
Vu qu'il s'agit d'un modèle de classification binaire, le test unitaire consistera à :
- faire prédire la variable cible par l'API du modèle 
- fournir au modèle les valeurs des attributs via un formulaire

Le micro-framework Web léger [_Flask_](https://flask.palletsprojects.com/en/3.0.x/quickstart/) prend en charge la gestion de requêtes et réponses REST. Cf. section [Serveur Web](#web_server)

In [1]:
import pandas as pd
import json
with open('../config.json') as file_object:
    dict_config = json.load(file_object)

# Tests avec _pytest_

## Installation de _pytest_

In [2]:
! pip install pytest

## Catalogue de tests
Des fichiers sont écrits depuis ce _Noteboook_ pour faciliter les mises à jour des paramètres de connexion tels que l'adresse et le port d'un serveur, par exemple 

### Test 1 de connexion et du format
Test de connexion au serveur et de vérification du format des données en entrée

In [3]:
dir_test_data = '../test_api/data/'
path_test_1 = './test_1.py'

#### Paramètres de connexion

In [4]:
ip_server           = dict_config['ip_host']
port_staging_server = dict_config['port_staging']
url_staging_server  = ip_server + ':' + port_staging_server + '/invocations'

#### Vérification de connexion

In [5]:
! curl -v "http://4.233.201.217:5677/version" 

*   Trying 4.233.201.217:5677...
* Connected to 4.233.201.217 (4.233.201.217) port 5677 (#0)
> GET /version HTTP/1.1
> Host: 4.233.201.217:5677
> User-Agent: curl/7.81.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: gunicorn
< Date: Mon, 01 Jul 2024 20:04:36 GMT
< Connection: close
< Content-Type: application/json
< Content-Length: 6
< 
* Closing connection 0
2.14.1

#### Contenu de _test_1.py_

In [6]:
str_test_1 = \
'''\
import requests
def test_connection():    
    print('__________Connection to API Server_____________')
    req_post = requests.post(   url     = 'http://ip_server:port_server/invocations', 
                                headers = {'Content-Type': 'application/json'}, 
                                data    = '{"dataframe_split": str_dataframe_split }' )
    # Evaluation of response
    is_connected = req_post.ok
    if is_connected : print('OK : Connected to API Server & validated input format')
    assert is_connected
'''

**Mise à jour du contenu**

On remplace les chaînes de caractères avec les valeurs des variables :
- 1. Connexion au serveur via requête POST 

In [7]:
str_test_1 = str_test_1 .replace('ip_server',   ip_server) \
                        .replace('port_server', port_staging_server)

- 2. Données avec un format valide (cf. [Deploy MLflow Model](https://mlflow.org/docs/latest/deployment/deploy-model-locally.html))

In [8]:
path_TP = '../modeling/data/out/X_TP.csv'   # True Positives dataset
df_TP = pd.read_csv(path_TP)
df_TP_sample_1 = df_TP.head(1)
dict_TP_sample_1 = df_TP_sample_1.to_dict(orient="split")
str_dataframe_split = str(dict_TP_sample_1).replace("'", '"').replace('True', 'true').replace('False', 'false')
str_dataframe_split

'{"index": [0], "columns": ["CODE_GENDER_M", "NAME_CONTRACT_TYPE_Cash_loans", "EXT_SOURCE_3", "NAME_EDUCATION_TYPE_Higher_education", "EXT_SOURCE_2", "FLAG_OWN_CAR", "NAME_EDUCATION_TYPE_Secondary_or_secondary_special"], "data": [[1, 1, 0.1176137317080569, 0, 0.7088758643589249, true, 1]]}'

In [9]:
str_test_1 = str_test_1.replace('str_dataframe_split', str_dataframe_split)

**Création du fichier**

In [10]:
with open(path_test_1, "w") as file_object:
    print(str_test_1, file=file_object)

In [11]:
!cat $path_test_1

import requests
def test_connection():    
    print('__________Connection to API Server_____________')
    req_post = requests.post(   url     = 'http://4.233.201.217:5677/invocations', 
                                headers = {'Content-Type': 'application/json'}, 
                                data    = '{"dataframe_split": {"index": [0], "columns": ["CODE_GENDER_M", "NAME_CONTRACT_TYPE_Cash_loans", "EXT_SOURCE_3", "NAME_EDUCATION_TYPE_Higher_education", "EXT_SOURCE_2", "FLAG_OWN_CAR", "NAME_EDUCATION_TYPE_Secondary_or_secondary_special"], "data": [[1, 1, 0.1176137317080569, 0, 0.7088758643589249, true, 1]]} }' )
    # Evaluation of response
    is_connected = req_post.ok
    if is_connected : print('OK : Connected to API Server & validated input format')
    assert is_connected



### Test 2 de prédiction
Comparaison des prédictions sur un jeu de 3 observations sensées être de classe 1

In [12]:
path_test_2 = './test_2.py'

In [13]:
str_test_2 = \
'''
import requests
import pandas as pd
import json

def test_predict():
    dict_inputs = str_dict_inputs                  # input observations      
    df_result_grid = pd.DataFrame(data=dict_inputs['data'], columns=dict_inputs['columns'])
    df_result_grid['target'] = str_li_targets      # expected output

    # Send input data to prediction API via POST request 
    req_post = requests.post(   url     = 'http://ip_server:port_server/invocations', 
                                headers = {'Content-Type': 'application/json'}, 
                                data    = '{"dataframe_split": str_dataframe_split }' )
    
    dict_predicted = json.loads(req_post.text)  # output predictions {'predictions': [1, 1, 1]}

    
    print(dict_predicted)

    
    df_result_grid['prediction'] = dict_predicted['predictions']
    print(df_result_grid.round(3).T)            # results as pandas

    # Compare predicted vs. expected values
    assert df_result_grid['prediction'].equals(df_result_grid['target'])
'''

**Mise à jour du contenu**

On remplace les chaînes de caractères avec les valeurs des variables :
- 1. Connexion au serveur via requête POST 

In [14]:
str_test_2 = str_test_2 .replace('ip_server',   ip_server) \
                        .replace('port_server', port_staging_server)

- 2. Jeu de 3 observations

In [15]:
nb_observations = 3
df_TP = pd.read_csv(path_TP) # True Positives
df_TP_sample_N = df_TP.tail(nb_observations)
df_TP_sample_N

,CODE_GENDER_M,NAME_CONTRACT_TYPE_Cash_loans,EXT_SOURCE_3,NAME_EDUCATION_TYPE_Higher_education,EXT_SOURCE_2,FLAG_OWN_CAR,NAME_EDUCATION_TYPE_Secondary_or_secondary_special
2291,1,1,0.474051,0,0.314709,False,1
2292,0,1,0.159520,1,0.322552,True,0
2293,0,1,0.126101,0,0.300759,False,1


In [16]:
str_dict_inputs = str(df_TP_sample_N.to_dict(orient="split")).replace("'", '"')
str_test_2 = str_test_2.replace('str_dict_inputs', str_dict_inputs)

In [17]:
str_dataframe_split = str_dict_inputs.replace('True', 'true').replace('False', 'false')
str_test_2 = str_test_2.replace('str_dataframe_split', str_dataframe_split)

- 3. Cibles attendues

In [18]:
li_targets = [1] * nb_observations
str_li_targets = str(li_targets)
str_li_targets

'[1, 1, 1]'

In [19]:
str_test_2 = str_test_2 .replace('str_li_targets', str_li_targets)

**Création du fichier**

In [20]:
with open(path_test_2, "w") as file_object:
    print(str_test_2, file=file_object)

In [21]:
!cat $path_test_2


import requests
import pandas as pd
import json

def test_predict():
    dict_inputs = {"index": [2291, 2292, 2293], "columns": ["CODE_GENDER_M", "NAME_CONTRACT_TYPE_Cash_loans", "EXT_SOURCE_3", "NAME_EDUCATION_TYPE_Higher_education", "EXT_SOURCE_2", "FLAG_OWN_CAR", "NAME_EDUCATION_TYPE_Secondary_or_secondary_special"], "data": [[1, 1, 0.4740512892789932, 0, 0.3147092250216505, False, 1], [0, 1, 0.1595195404777181, 1, 0.3225519037906617, True, 0], [0, 1, 0.1261005588362325, 0, 0.3007587165928464, False, 1]]}                  # input observations      
    df_result_grid = pd.DataFrame(data=dict_inputs['data'], columns=dict_inputs['columns'])
    df_result_grid['target'] = [1, 1, 1]      # expected output

    # Send input data to prediction API via POST request 
    req_post = requests.post(   url     = 'http://4.233.201.217:5677/invocations', 
                                headers = {'Content-Type': 'application/json'}, 
                                data    = '{"dataframe_split"

## Exécution et résultats des tests

In [22]:
!python -m pytest --import-mode=append -rA ../test_api/

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.2.2, pluggy-1.5.0
rootdir: /home/jvisa4031/project_7/test_api
plugins: anyio-4.4.0, Faker-25.5.0
collected 2 items                                                              

test_1.py .                                                              [ 50%]
test_2.py .                                                              [100%]

==================================== PASSES ====================================
_______________________________ test_connection ________________________________
----------------------------- Captured stdout call -----------------------------
__________Connection to API Server_____________
OK : Connected to API Server & validated input format
_________________________________ test_predict _________________________________
----------------------------- Captured stdout call -----------------------------
{'predictions': [1, 1, 1]}
   

**Observation** : les résultats des tests unitaires sont positifs

# Tests via _curl_

In [23]:
import subprocess

Récupération de la liste d'attributs

In [24]:
with open(dir_test_data + 'li_features.txt') as file_object:
    str_li_features = file_object.read()
str_li_features = str_li_features.replace('\'', '"').replace('\n', '')
str_li_features

'["CODE_GENDER_M", "NAME_CONTRACT_TYPE_Cash_loans", "EXT_SOURCE_3", "NAME_EDUCATION_TYPE_Higher_education", "EXT_SOURCE_2", "FLAG_OWN_CAR", "NAME_EDUCATION_TYPE_Secondary_or_secondary_special"]'

In [25]:
def get_single_prediction(str_values, str_expected) :
    shell_command = 'curl -d \'{"dataframe_split": { "columns": ' + str_li_features \
        + ', "data": [' + str_values + ']}}\' -H \'Content-Type: application/json\' -X POST ' \
        + url_staging_server + ' --no-progress-meter'
    print('Shell command :', shell_command, '\n')
    print(subprocess.getoutput(shell_command))
    print('expected :', str_expected, 'for features values:', str_values)

In [26]:
str_values_TP_sample_1 = str(dict_TP_sample_1['data'][0]).replace('True', 'true').replace('False', 'false')
str_values_TP_sample_1

'[1, 1, 0.1176137317080569, 0, 0.7088758643589249, true, 1]'

In [27]:
get_single_prediction(str_values_TP_sample_1, '1')

Shell command : curl -d '{"dataframe_split": { "columns": ["CODE_GENDER_M", "NAME_CONTRACT_TYPE_Cash_loans", "EXT_SOURCE_3", "NAME_EDUCATION_TYPE_Higher_education", "EXT_SOURCE_2", "FLAG_OWN_CAR", "NAME_EDUCATION_TYPE_Secondary_or_secondary_special"], "data": [[1, 1, 0.1176137317080569, 0, 0.7088758643589249, true, 1]]}}' -H 'Content-Type: application/json' -X POST 4.233.201.217:5677/invocations --no-progress-meter 

{"predictions": [1]}
expected : 1 for features values: [1, 1, 0.1176137317080569, 0, 0.7088758643589249, true, 1]


# Tests via interface Web d'API

### Accès en ligne

In [28]:
port_web_server = dict_config['port_flask']
print('Website URL =' , 'http://' + ip_server + ':' + port_web_server)

Website URL = http://4.233.201.217:6543


### Arborescence de fichiers

In [29]:
! find ../website | grep -v "checkpoint" | sed -e "s/[^\/]*\//  |/g" -e "s/|\([^ ]\)/└── \1/"

  └── website
  |  └── static
  |  |  └── css
  |  |  |  └── style.css
  |  └── web_router.py
  |  └── __pycache__
  |  |  └── web_router.cpython-310.pyc
  |  └── templates
  |  |  └── api_observations.html
  |  |  └── form.html
  |  |  └── report.html
  |  |  └── index.html
  |  |  └── result.html
  |  |  └── report_simulation.html
  |  └── instructions_WebServer.md


### Fichiers _HTML_

In [30]:
import IPython
dir_website = '../website/'
dir_html = dir_website + 'templates/'

#### Index  
Fichier d'accueil

In [31]:
file_html_index = dir_html + 'index.html'
!cat $file_html_index


<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <a href="/form/"><h4>Formulaire</h4></a>
       <a href="/report/"><h4>Rapport Evidently</h4></a>
       <a href="/report_simul/"><h4>Rapport Evidently de simulation</h4></a>
       <img src="https://miro.medium.com/v2/resize:fit:1200/0*5llOZkUFa4KHgDMQ.jpg">
    </body>
</html>



In [32]:
IPython.display.IFrame(file_html_index, width=700, height=350) 

#### Formulaire  
Formulaire de saisie des atttributs

In [33]:
path_form = dir_html + 'form.html'

In [34]:
str_form = \
'''\
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
        <h1>API du modèle de classification binaire</h1>       
        <h2>Formulaire de saisie d'attributs</h2>       
        
        <form method="post" action="{{ url_for('result') }}">
            <div>
            	<label for="port">Server:</label>
            	<select id="port" name="port">
                		<option value="5677"> Staging    </option>
                		<option value="5678"> Production </option>
            	</select>
            </div>
            <br>
            <table> \
'''

In [35]:
li_features = eval(str_li_features)
li_features

['CODE_GENDER_M',
 'NAME_CONTRACT_TYPE_Cash_loans',
 'EXT_SOURCE_3',
 'NAME_EDUCATION_TYPE_Higher_education',
 'EXT_SOURCE_2',
 'FLAG_OWN_CAR',
 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special']

In [36]:
with open(dir_test_data + 'li_types.txt') as file_object:
    str_li_types = file_object.read()
str_li_types = str_li_types.replace('[', '["').replace(', ', '", "').replace(']\n', '"]')
li_types = eval(str_li_types)
li_types

['long', 'double', 'double', 'long', 'long', 'long', 'long']

In [37]:
def get_div_per_feature(idx, str_feature_i, str_type_i) :
    str_div_i = \
    '''
                <tr>
                    <td><label for="input_str_idx">str_feature_i :</label></td>
                    <td><input name="str_feature_i" id="input_str_idx" type="number" step="any" value="1"></td>
                    <td>str_type_i</td>
                </tr> \
    '''
    return str_div_i.replace('str_idx',       str(idx))      \
                    .replace('str_feature_i', str_feature_i) \
                    .replace('str_type_i',    str_type_i)

In [38]:
for idx, (feature_i, type_i) in enumerate(zip(li_features, li_types)) :
    str_form += get_div_per_feature(idx + 1, feature_i, type_i)

In [39]:
str_form += \
'''
            </table>
            <br>
            <div>
                <button>Predict</button>
            </div>
        </form>
        <br>
        <p>Observations de Classe 1 (mauvais emprunteur) - TP (vrai positif):</p>
        str_pandas_as_html_TP
        <p>Observations de Classe 0 (bon emprunteur) - TN (vrai négatif):</p>
        str_pandas_as_html_TN

    </body>
</html>
'''

In [40]:
df_TP_sample_10 = pd.read_csv(path_TP).sample(10, random_state=0)
df_TP_sample_10.T

,2045,443,674,1193,1027,405,1878,794,2281,535
CODE_GENDER_M,1,1,1,0,0,0,0,1,0,0
NAME_CONTRACT_TYPE_Cash_loans,1,1,1,1,1,1,1,1,1,1
EXT_SOURCE_3,0.450747,0.297087,0.515495,0.377404,0.300108,0.304672,0.155689,0.167408,0.15664,0.121408
NAME_EDUCATION_TYPE_Higher_education,0,0,0,0,0,0,0,0,1,0
EXT_SOURCE_2,0.093445,0.422152,0.315975,0.517697,0.250257,0.277667,0.542497,0.478596,0.069169,0.171423
FLAG_OWN_CAR,False,False,False,False,False,True,False,True,True,False
NAME_EDUCATION_TYPE_Secondary_or_secondary_special,1,0,1,1,0,1,1,1,0,1


In [41]:
str_pandas_as_html_TP = df_TP_sample_10.T.to_html(justify='center', border=0, max_rows=10) #, 
                                #float_format=lambda x: '%.10f' % x)
str_form = str_form .replace('str_pandas_as_html_TP', str_pandas_as_html_TP) \
                    .replace('.000000', '') #('.0000000000', '')

In [42]:
path_TN = '../modeling/data/out/X_TN.csv'   # True Negatives dataset
df_TN_sample_10 = pd.read_csv(path_TN).sample(10, random_state=0)
df_TN_sample_10.T

,14067,10204,18848,13534,3085,31366,24707,19330,12277,30699
CODE_GENDER_M,0,0,0,0,1,0,0,1,1,1
NAME_CONTRACT_TYPE_Cash_loans,1,1,0,1,1,1,1,1,1,1
EXT_SOURCE_3,0.609276,0.725276,0.461482,0.403142,0.56206,0.715103,0.394495,0.652897,0.654529,0.786267
NAME_EDUCATION_TYPE_Higher_education,0,0,0,0,0,1,1,0,0,1
EXT_SOURCE_2,0.568989,0.640071,0.346802,0.701575,0.581964,0.734849,0.679545,0.67705,0.531891,0.621099
FLAG_OWN_CAR,False,True,False,True,True,False,False,True,True,False
NAME_EDUCATION_TYPE_Secondary_or_secondary_special,1,1,1,1,1,0,0,1,0,0


In [43]:
str_pandas_as_html_TN = df_TN_sample_10.T.to_html(justify='center', border=0, max_rows=10)
str_form = str_form .replace('str_pandas_as_html_TN', str_pandas_as_html_TN) \
                    .replace('.000000', '') 

In [44]:
with open(path_form, 'w') as file_object:
    print(str_form, file=file_object)

In [45]:
!cat $path_form

<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
        <h1>API du modèle de classification binaire</h1>       
        <h2>Formulaire de saisie d'attributs</h2>       
        
        <form method="post" action="{{ url_for('result') }}">
            <div>
            	<label for="port">Server:</label>
            	<select id="port" name="port">
                		<option value="5677"> Staging    </option>
                		<option value="5678"> Production </option>
            	</select>
            </div>
            <br>
            <table> 
                <tr>
                    <td><label for="input_1">CODE_GENDER_M :</label></td>
                    <td><input name="CODE_GENDER_M" id="input_1" type="number" step="any" value="1"></td>
                    <td>long</td>
                </tr>     
           

In [46]:
IPython.display.IFrame(path_form, width=700, height=350) 

#### Résultat
Page de résultat de prédiction par l'API du modèle

In [47]:
path_result = dir_html + 'result.html'

In [48]:
str_result = \
'''\
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page de prédiction du modèle</h2> 
       <h3>Valeur prédite = {{target_value['predictions']}}</h3>
       <p>Pour les valeurs saisies des attributs suivants :</p>
       <table>
'''

In [49]:
li_features

['CODE_GENDER_M',
 'NAME_CONTRACT_TYPE_Cash_loans',
 'EXT_SOURCE_3',
 'NAME_EDUCATION_TYPE_Higher_education',
 'EXT_SOURCE_2',
 'FLAG_OWN_CAR',
 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special']

In [50]:
def get_div_per_feature(idx, str_feature_i) :
    str_div_i = \
    ''' 
            <tr>
                <td>str_feature_i</td><td>{{features['str_feature_i']}}</td>  
            </tr> \
    '''
    return str_div_i.replace('str_idx',       str(idx))    \
                    .replace('str_feature_i', str_feature_i)                        

In [51]:
for idx, feature_i in enumerate(li_features) :
    str_result += get_div_per_feature(idx + 1, feature_i)

In [52]:
str_result += \
'''
       </table>
       <br>
       <a href="/">Retour à l'accueil</a>
       <a href="/form/">Formulaire</a>
    </body>
</html>
'''

In [53]:
with open(path_result, "w") as file_object:
    print(str_result, file=file_object)

In [54]:
!cat $path_result

<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
        <link rel="stylesheet" href="{{ url_for('static', filename= 'css/style.css') }}">
        <title>ML model API</title>
    </head>
    <body>
       <h1>API du modèle de classification binaire</h1>      
       <h2>Page de prédiction du modèle</h2> 
       <h3>Valeur prédite = {{target_value['predictions']}}</h3>
       <p>Pour les valeurs saisies des attributs suivants :</p>
       <table>
 
            <tr>
                <td>CODE_GENDER_M</td><td>{{features['CODE_GENDER_M']}}</td>  
            </tr>      
            <tr>
                <td>NAME_CONTRACT_TYPE_Cash_loans</td><td>{{features['NAME_CONTRACT_TYPE_Cash_loans']}}</td>  
            </tr>      
            <tr>
                <td>EXT_SOURCE_3</td><td>{{features['EXT_SOURCE_3']}}</td>  
            </tr>      
            <tr>
                <td>NAME_EDUCATION_TYPE_Higher_education</td><td>{{features['NAME_EDUCATION_TYPE_Higher_education']

In [55]:
IPython.display.IFrame(path_result, width=700, height=350) 

#### Style

In [56]:
dir_style = '../website/static/css/'

In [57]:
file_style = dir_style + 'style.css'
!cat $file_style

html {
    text-align: center;
    font-family: Arial;
}
h1 {
    border: 2px #eee solid;
    color: brown;
    text-align: center;
    padding: 10px;
}
h3 {
    color: darkred;
}
table { 
    margin-left: auto;
    margin-right: auto;
    font-size: 12px;
}
tr:nth-child(odd) {
    background-color: #e6f7ff;
}
tr:nth-child(even) {
    background-color: #b3e6ff;
}

### Fichiers _Python_

#### Routeur Web  
Un fichier en Pyhton couvre la gestion REST

In [58]:
web_router = dir_website + 'web_router.py'
!cat $web_router

import pandas as pd
import json
import subprocess
import requests
from flask import Flask, render_template, request
from mlflow import sklearn as skl
from sklearn.metrics import recall_score, roc_auc_score

# **************************************************** FUNCTIONS ************************
def run_shell(command) :
    shell_process = subprocess.run([command], shell=True, capture_output=True, text=True)
    return str(shell_process.stdout) + str(shell_process.stderr)
def pull() :
    dir_root = '../'
    str_command_pull = 'cd ' + dir_root + ' ; git pull origin main'
    str_output = run_shell(str_command_pull)
    return 'Git Pull Model:===============\n' + str_output
def restart(str_environment) : 
    if str_environment == 'staging'    : dir_model, port = '../api/staging_model/',    '5677'
    if str_environment == 'production' : dir_model, port = '../api/production_model/', '5678' 
    str_command_serve = 'mlflow models serve -m ' + dir_model + ' -p ' + port + ' -h 0.0.0.0 --n

## Serveur _Web_

<a id="web_server"></a>
<hr>

### Lancement

In [59]:
shell_command = 'nohup flask --app ' + web_router + ' run -h 0.0.0.0 -p ' + port_web_server + \
    ' --debug > ./flask_app.log 2>&1 &'
print('shell_command =', shell_command)
get_ipython().system_raw(shell_command) # run model API in background

shell_command = nohup flask --app ../website/web_router.py run -h 0.0.0.0 -p 6543 --debug > ./flask_app.log 2>&1 &


### Vérifications
#### Log

In [60]:
! tail ./flask_app.log

#### Processus
Liste des processus (2)

In [63]:
li_ps = !ps aux | grep "flask" | grep -v "grep" | awk '{print $2}' 
!ps aux | grep "flask" | grep -v "grep" 

jvisa40+   35256  0.0  1.2 678880 199568 ?       Sl   08:49   0:02 /home/jvisa4031/environments_folder/my_env/bin/python3 /home/jvisa4031/environments_folder/my_env/bin/flask --app ../website/web_router.py run -h 0.0.0.0 -p 6543 --debug
jvisa40+   35266  1.0  1.7 1952212 281204 ?      Sl   08:49   7:10 /home/jvisa4031/environments_folder/my_env/bin/python3 /home/jvisa4031/environments_folder/my_env/bin/flask --app ../website/web_router.py run -h 0.0.0.0 -p 6543 --debug


Arrêt des processus

In [64]:
for ps_i in li_ps : 
    #! kill -9 $ps_i
    pass